In [ ]:
import bokeh

In [ ]:

import math
def InitialDataReducedMass(orbitalangle,r0):
    phi=orbitalangle
    radiusseparation=r0 #separation one hundred
    eccentricity=np.zeros(2)
    mass=np.ones(2)
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    masses=mass #*masssun (natural units)
    return phi, radiusseparation,eccentricity, masses
    

In [ ]:
def getxyuveqreducedmass(initdat): #using reduced mass cooordinate system
    phi,radiusseparation,eccentricity, masses=initdatreducedmass
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    x0=np.zeros(2)
    y0=np.zeros(2)
    x0[1]=radiusseparation*np.cos(phi)*metersperAU
    y0[1]=radiusseparation*np.sin(phi)*metersperAU
    if phi==0:
        print("zero")
        x0[1]=radiusseparation
        y0[1]=0
    if phi==math.pi:
        print("pi")
        x0[1]=-radiusseparation
        y0[1]=0
    z0=np.zeros(2)
    print("x0", x0)
    print("y0", y0)
    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2) #*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    masstotal=np.sum(masses)
    reducedmass[0]=masstotal
    print(masses)
    for i in np.arange(1):
        j=(i+1)%2 #reverse masses
        reducedmass[i+1]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    metersperAU=1 #natural units
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        #rphys is r in the reduced mass system
        rphys[i]=radiusseparation #*metersperAU*reducedmass[i]/masses[i]
    x0[0]=0.0
    x0phys=rphys*x0/radiusseparation
    x0phys[0]=0.0
    y0phys=rphys*y0/radiusseparation
    z0phys=np.zeros(2)
    r0=np.sqrt(x0**2+y0**2)
    print(rphys)
    #G=1
    Gconstant=1
    F=(Gconstant*masstotal**2/(2*rphys)**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*masstotal/(rphys))
    print(v)
    print("r0", r0)
    ux0=-v*y0/radiusseparation #[1]
    uy0=v*x0/radiusseparation #[1]
        
    #ux0=-v*y0/r0
    #uy0=v*x0/r0 #initial data in y only
    #evolve in plane only
    #circular orbit
    uz0=np.zeros(2)
    
    a=Gconstant*masstotal/((2*rphys)**2)
    ax0=-a*x0/radiusseparation
    ay0=-a*y0/radiusseparation
    az0=np.zeros(2)
    
    statevec=[]
    avec=[]
    for i in np.arange(len(x0phys)):
        stateveci=np.array([x0phys[i],y0phys[i],z0phys[i],ux0[i],uy0[i],uz0[i]])
        aveci=np.array([ax0[i],ay0[i],az0[i]])
        statevec.append(stateveci)
        avec.append(aveci)
    statevecnp=np.array(statevec)
    avecnp=np.array(avec)
    return masstotal, reducedmass,statevecnp,avecnp

In [ ]:
import random,numpy as np
initdatreducedmass=InitialDataReducedMass(0,50)

In [ ]:
xyuvaeqreducedmass=getxyuveqreducedmass(initdatreducedmass)
print(xyuvaeqreducedmass)
masstotal, reducedmass,xvec,avec=xyuvaeqreducedmass

In [ ]:
xvec[:,2]

In [ ]:
axii=np.zeros(len(xvec[:,0]))
rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
rii

In [ ]:
import OrbitDiffEq

In [ ]:
mtotal,reducedmass,xvec,avec=xyuvaeqreducedmass
ODEeq= OrbitDiffEq.OrbitDiffEq(reducedmass,xvec,avec,0.0)
ODEeq.print2D()
ODEeq.print2D()

In [ ]:
dt=1. #natural units #*31556926 #seconds per year
numsteps=20000
mtotal, reducedmass,xvec0,avec0=xyuvaeqreducedmass
ODEeq= OrbitDiffEq.OrbitDiffEq(reducedmass,xvec0,avec0,0.0)
ODEeq.print2D()
t=0.0
star1x=[]
star2x=[]
star1a=[]
star2a=[]
times=[]
for i in np.arange(1,numsteps):
    reducedmass,xvec,avec,t=ODEeq.timestepRK4ODE(i,dt)
    if i< 10:
        ODEeq.print2D()
    star1x.append(xvec[0])
    star2x.append(xvec[1])
    star1a.append(avec[0])
    star2a.append(avec[1])
    times.append(t)



In [ ]:
npstar2x=np.array(star2x)
npstar1x=np.array(star1x)
npstar2a=np.array(star2a)
npstar1a=np.array(star1a)
nptimes=np.array(times)

In [ ]:
max(npstar2x[1,:])

Orbit doesn't go out to 53

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="Circular orbit, RK4", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(npstar1x[:,0],npstar1x[:,1], size=15, line_color="purple",fill_color="purple", fill_alpha=0.5), #legend="M")
p.line(npstar2x[:,0],npstar2x[:,1], line_color="blue"), #legend="mu")


#p.legend.location = "bottom_left"
#p.legend.click_policy="hide"

show(p) # show the

%%html
<a href="https://github.com/sdorsher/ManyBodyProblem/blob/master/reduced%20mass%20circular%20orbit%20rk4.png"> Circular orbit in the reduced mass system </a> 
Note that the central heavier mass is stationary in this reference frame with the smaller lighter mass orbits about it in a circular orbit. 

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="X-acceleration, Circular orbit, RK4", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(times,npstar1a[:,0], size=1, line_color="purple",fill_color="purple", fill_alpha=0.5) #legend="M")
p.circle(times,npstar2a[:,0], size=1, line_color="blue") #legend="mu")

#p.legend.location = "bottom_left"
#p.legend.click_policy="hide"

show(p) # show the

%%html
<a href="https://github.com/sdorsher/ManyBodyProblem/blob/master/reduced%20mass%20acceleration%20circular%20RK4.png">Acceleration in the reduced mass reference frame for a circular orbit </a> 

In the reduced mass reference frame, the acceleration of the larger mass is zero and the acceleration of the x componenet of the smaller mass is sinusiodal. 

In [ ]:
def RunFirstOrbit(dt,initcondition,printoutput):
    masstotal,mass0,xvec, avec=initcondition
    x0=xvec[1,0]
    y0=xvec[1,1]
    lastvalue=[1,1]
    thisvalue=[xvec[1,0],xvec[1,1]]
    i=1
    t=0
    while not (lastvalue[0]>0 and lastvalue[1]<0 and thisvalue[0]> 0 and thisvalue[1]>=0):
        reducedmass,xvec,avec,t=ODEeq.timestepRK4ODE(i,dt)
        #if printoutput:
        #    print(xvec[1,0])
        #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
        #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
        #print(ax)
        xy=[xvec[1,0],xvec[1,1]]
        lastvalue,thisvalue=thisvalue, xy
        #lastvalue=thisvalue
        #thisvalue[0]=x[1]
        #thisvalue[1]=y[1]
        #thisvalue[0]=x[1]
        #thisvalue[1]=y[1]
        if printoutput:
            print(xvec[1,0],xvec[1,1],xy,lastvalue,thisvalue)
        i+=1
    #slope=(thisvalue[1]-lastvalue[1])/(thisvalue[0]-lastvalue[0])
    #xcrossing=1./slope*(0.-lastvalue[1])+lastvalue[0] #(y1-y2)=m(x1-x2)
    xcrossing=(lastvalue[0]+thisvalue[0])/2.
    print(dt, thisvalue[0], thisvalue[1], lastvalue[0], lastvalue[1], x0, y0,xcrossing,np.abs(x0-xcrossing))
    #interpolate where it crosses zero
    #slope=(thisvalue[1]-lastvalue[1])/(thisvalue[0]-lastvalue[0])
    #xcrossing=1./slope*(0.-lastvalue[1])+lastvalue[0] #(y1-y2)=m(x1-x2)
    return np.abs(x0-xcrossing)

In [ ]:
dt=.1 #*31556926 #seconds per year
numsteps=2000
mtotal,mass0,xvec,avec=xyuvaeqreducedmass
L0arr=[]
dtarr=[]
dtinit=0.125/4.
for j in np.arange(1):
    dt=(2**j)*dtinit
    dtarr.append(dt)
    L0=RunFirstOrbit(dt,xyuvaeqreducedmass,True)
    L0arr.append(L0)
print(dtarr, L0arr)

In [ ]:
xdat=np.log2(np.array(dtarr))
ydat=np.log2(np.array(L0arr))

In [ ]:
fitparam=np.polyfit(xdat,ydat,1)

In [ ]:
fitparam

In [ ]:
errfn=np.poly1d(fitparam)

In [ ]:
errfn

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="L0 error after one orbit, Circular, Vector, RK4", x_axis_label="dt", y_axis_label="L0", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha

p.circle(np.log2(np.array(dtarr)),np.log2(np.array(L0arr)), line_color="blue", legend="L0")
p.line(xdat,errfn(xdat),line_color="green", legend="Fit, dt^4.1")
p.legend.location = "bottom_right"
p.legend.click_policy="hide"

show(p) # show the

%%html 
<a href="https://github.com/sdorsher/ManyBodyProblem/blob/master/L0%20error%20Circular%20Vector%20RK4%20dt%204p1.png"> L0 error after one circular orbit in the reduced mass sytem </a>

L0 error is the absolute value of the difference between the x coordinate at t=0 and the x coordinate after one orbit. I have plotted the behavior of the L0 error as a function of the timestep dt. For the RK4 numerical differential equation solution method, the error should go as dt^4 or better. It doe, since it goes as dt^4.1. 